# Um problema de otimização.

Neste exercício propomos a busca pelo solução do seguinte problema

Um veiculo possui um sistema de controle capaz de estabelecer a trajetória que o mesmo deve seguir. O mecanismo define a coordenada $y$ do veiculo como função da coordenada $x$, para um percurso que vai desde $x = 0$ até $x = 10$. O sistema de controle é uma caixa preta, ou seja, um mecanismo desconhecido que, dada uma entrada produz uma sequencia de comandos que leva o veículo por um determinado trajeto. A entrada da caixa preta é uma sequencia de 16 bits, o que pode ser interpretado como se o mecanismo tivesse 65536 rotas pre programadas. A saída são dois NumPy arrays contendo as coordenadas $x$ e $y$ da trajetória. 

Os desenvolvedores querem descobrir qual rota percorre um trajeto específico ou um que seja o mais próximo possível desse trajeto. Com esta finalidade foi definida uma função heuristic que retorna o RMSE. Procure como se calcula o erro médio quadrático (MSE - mean squared error) e a raiz do erro médio quadrático (RMSE - root mean squared error) para entender como esta função heurística funciona. Para isto pode ser utilizado o método de força bruta e testar cada uma das entradas da caixa preta a procura do menor valor da heurística.  

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from caixaPreta import heuristic, bBoxEx, bin2list, list2bin

In [4]:
# força bruta
min = heuristic(bBoxEx(1))
rota = 1
for i in range(2, 2**16):
    h = heuristic(bBoxEx(i)) 
    if  h < min:
        min = h
        rota = i
print("Força bruta: ", min, rota)

Força bruta:  6.172569552785352e-16 18465


Podemos então utilizar alguns dos métodos que estudamos em sala de aula para este tipo de busca. Para começar precisamos gerar um stado inicial. 

In [21]:
current = np.random.randint(low=1, high=2**16)
print(f"Estado inicial aleatório: {current}")
print(f"com heurística: {heuristic(bBoxEx(current))}")

Estado inicial aleatório: 27307
com heurística: 2.669119711968321


Os estados vizinhos de um estado são aqueles em que apenas um único bit é mudado de cada vez

In [9]:
def next_state(state):
    next_states = []
    l = bin2list(state)
    for i in range(16):
        nl = l.copy()
        nl[i] = 1 - nl[i]
        next_states.append(list2bin(nl))
    return next_states

In [25]:
neighbors = next_state(current)
print(f"Estado vizinhos de {current}:  {neighbors}")
print(current, bin2list(current), heuristic(bBoxEx(current)))
for n in neighbors:
    print(n, bin2list(n), heuristic(bBoxEx(n)))

Estado vizinhos de 27307:  [60075, 10923, 19115, 31403, 25259, 28331, 26795, 27563, 27179, 27371, 27275, 27323, 27299, 27311, 27305, 27306]
27307 [0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1] 2.669119711968321
60075 [1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1] 2.306708156142998
10923 [0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1] 3.9884832379182775
19115 [0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1] 0.5563720698299199
31403 [0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1] 2.2533152506711494
25259 [0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1] 3.539924048714635
28331 [0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1] 3.203762709498443
26795 [0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1] 2.934300615177598
27563 [0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1] 3.1170798215342588
27179 [0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1] 2.971357579567107
27371 [0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1] 3.056755983958217
27275 [0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1

Se ordenar alista utilizando a função heurística teremos que:

In [26]:
neighbors = sorted(next_state(current), key = lambda x:heuristic(bBoxEx(x)))
print(current, bin2list(current), heuristic(bBoxEx(current)))
for n in neighbors:
    print(n, bin2list(n), heuristic(bBoxEx(n)))

27307 [0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1] 2.669119711968321
19115 [0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1] 0.5563720698299199
27311 [0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1] 1.7627679776673681
27323 [0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1] 1.8529781802462153
27299 [0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1] 2.078101009649483
27305 [0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1] 2.16933461836946
31403 [0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1] 2.2533152506711494
60075 [1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1] 2.306708156142998
26795 [0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1] 2.934300615177598
27179 [0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1] 2.971357579567107
27371 [0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1] 3.056755983958217
27563 [0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1] 3.1170798215342588
28331 [0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1] 3.203762709498443
25259 [0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0

Podemos pensar então em métodos que estudando, le,brando que cada ponto (estado) da paisagem possui uma “elevação”, definida pelo valor da função objetivo. Se a elevação corresponde a uma função objetivo, então o objetivo é encontrar o pico mais alto – __um máximo global__ – e chamamos o processo de __subida de encosta__. Se a elevação corresponde ao custo, então o objectivo é encontrar o vale mais baixo – __um mínimo global__ – e chamamos-lhe __descida de gradiente__. O problema, neste caso, é de descida do gradiente. 
Implemente:
* Busca da descida de gradiente;
* Busca da descida de gradiente com movimentos laterais;
* Busca da descida estocástica; (escolhe aleatoriamente entre os movimentos de descida; a probabilidade de seleção pode variar com a inclinação do movimento descendente;
* Busca da descida com reinício aleatório; (conduz uma série de buscas descendente a partir de estados iniciais gerados aleatoriamente, até que um objetivo seja encontrado. Lembre que neste caso o objetivo pode estar indefinido)
* Busca da descida com reinício aleatório com movimentos laterais;
* Têmpera simulada (Simulated annealing); (selecione, implemente e teste pelo menos duas funções de schedule)
* Busca de feixe local;
* Busca de feixe local estocástica;(escolhe sucessores com probabilidade proporcional ao valor do sucessor)



